# Using the ESCALATEClient REST API Client


In [1]:
#factor this out when it's imported into the client

import json
import os

from pprint import pprint
import pandas as pd
import requests

class ESCALATEClient():
    """ESCALATE API Client"""
    
    def __init__(self, base_url, username, password):
        self.base_url = base_url
        self.username = username
        self.password = password
        self._token = None
        self._is_logged_in = False
        self._login()
        
    def _login(self):
        r_login = requests.post(f'{self.base_url}/api/login', 
                                data={'username': self.username, 
                                      'password': self.password})
        self._token = r_login.json()['token']
        self._token_header = {'Authorization': f'Token {self._token}'}
        self._is_logged_in = True
        
    def get(self, 
            endpoint='', 
            data=None, 
            parse_json=True, 
            content_type='application/json'):
        """Make GET request with `data` to `endpoint` in ESCALATE API
    
        return: (dict|list|requests.Response), bool
        """
        data = {} if data is None else data
        r = requests.get(f'{self.base_url}/api/{endpoint}',
                         params=data, 
                         headers={**self._token_header, 
                                  'content-type': content_type})
        if r.ok: 
            print('GET: OK')
        else: 
            print('GET: FAILED')
        
        if r.ok and parse_json:
            resp_json = r.json()        
            # handle cases: no vs one vs many results
            count = resp_json.get('count')
            if count is None or count == 0:
                return r.json()
            elif count == 1: 
                print('Found one resource, returning dict')
                return resp_json['results'][0]
            elif count >= 1: 
                print(f"Found {resp_json['count']} resources, returning list of dicts)")
                return r.json()['results']
        print('Returning response object')   
        return r
        
    def post(self, 
             endpoint, 
             data, 
             content_type='application/json'):
        """POST `data` to `endpoint`in ESCALATE API using `content_type`
        return: (dict|requests.Response), bool
        """
        
        if not self._is_logged_in:
            raise ValueError("Not logged in: cannot post")
        
        r = requests.api.post(
            f'{self.base_url}/api/{endpoint}', 
            data=json.dumps(data), 
            headers={**self._token_header,
                     'content-type': content_type})
        print(r)
        if r.ok: 
            print('POST: OK, returning new resource dict')
            return r.json()
        print('POST: FAILED, returning response object')
        return r
    
    def put(self, url=None, endpoint=None, resource_id=None, data=None):
        """Update a complete resource
        Either provide a url or an endpoint and resource id
        """
        if not ((url is not None) or (endpoint is not None and resource_id is not None)): 
            raise ValueError("Must specify either url or endpoint and resource_id")
            
        if url is None: 
            url = f'{self.base_url}/api/{endpoint}/{resource_id}' 
        r = requests.api.put(url, data, headers=self._token_header)
        return r
    
    def patch(self, url=None, endpoint=None, resource_id=None, data=None):
        """Update parts of a resource
        Either provide a url or an endpoint and resource id
        """
        if not ((url is not None) or (endpoint is not None and resource_id is not None)): 
            raise ValueError("Must specify either url or endpoint and resource_id")
            
        if url is None: 
            url = f'{self.base_url}/api/{endpoint}/{resource_id}' 
        r = requests.api.patch(url, data, headers=self._token_header)
        return r
        
    def delete(self, url=None, endpoint=None, resource_id=None):
        """Delete a resource
         Either provide a url or an endpoint and resource id
        """
        if not ((url is not None) or (endpoint is not None and resource_id is not None)): 
            raise ValueError("Must specify either url or endpoint and resource_id")
        if url is None: 
            url = f'{self.base_url}/api/{endpoint}/{resource_id}' 
        r = requests.api.delete(url, headers=self._token_header)
        return r
    
    def search(self, 
        endpoint='',
        related_ep=None, #for cross-searches
        search_field='',
        criteria= '',
        data=None, #must be a list
        exact=False,
        negate=False,
        parse_json=True, 
        content_type='application/json'):
    
        if negate==False:

            if exact==True:
                if data == None:
                    '''Returns all fields for exact match'''
                    if related_ep == None:

                        r = requests.get(f'{self.base_url}/api/{endpoint}/?{search_field}={criteria}', 
                                     headers={**self._token_header, 
                                              'content-type': content_type})
                    else: #cross-search 
                        r = requests.get(f'{self.base_url}/api/{endpoint}/?{related_ep}__{search_field}={criteria}', 
                                     headers={**self._token_header, 
                                              'content-type': content_type})

                else:
                    '''Returns requested field(s) for exact match'''
                    i=0
                    data_string=''
                    while i<len(data)-1:
                        data_string+=data[i]+','
                        i+=1
                    data_string+=data[i]

                    if related_ep ==None:

                        r = requests.get(f'{self.base_url}/api/{endpoint}/?{search_field}={criteria}&fields={data_string}', 
                                         headers={**self._token_header, 
                                                  'content-type': content_type}) 
                    else: #cross-search

                        r = requests.get(f'{self.base_url}/api/{endpoint}/?{related_ep}__{search_field}={criteria}&fields={data_string}', 
                                         headers={**self._token_header, 
                                                  'content-type': content_type}) 

            else:


                if data == None:
                    '''Containment test; returns all fields'''

                    if related_ep == None:
                        r = requests.get(f'{self.base_url}/api/{endpoint}/?{search_field}__icontains={criteria}', 
                                         headers={**self._token_header, 
                                                  'content-type': content_type}) 
                    else: #cross-search
                        r = requests.get(f'{self.base_url}/api/{endpoint}/?{related_ep}__{search_field}__icontains={criteria}', 
                                         headers={**self._token_header, 
                                                  'content-type': content_type})

                else:
                    '''Containment test; returns requested field(s)'''
                    i=0
                    data_string=''
                    while i<len(data)-1:
                        data_string+=data[i]+','
                        i+=1
                    data_string+=data[i]

                    if related_ep==None:
                        r = requests.get(f'{self.base_url}/api/{endpoint}/?{search_field}__icontains={criteria}&fields={data_string}', 
                                         headers={**self._token_header, 
                                                  'content-type': content_type}) 
                    else: #cross-search
                        r = requests.get(f'{self.base_url}/api/{endpoint}/?{related_ep}__{search_field}__icontains={criteria}&fields={data_string}', 
                                             headers={**self._token_header, 
                                                      'content-type': content_type})
        else:
        #negations

            if exact==True:
                if data == None:
                    '''Returns all fields for exact match'''
                    if related_ep == None:

                        r = requests.get(f'{self.base_url}/api/{endpoint}/?{search_field}!={criteria}', 
                                     headers={**self._token_header, 
                                              'content-type': content_type})
                    else: #cross-search 
                        r = requests.get(f'{self.base_url}/api/{endpoint}/?{related_ep}__{search_field}!={criteria}', 
                                     headers={**self._token_header, 
                                              'content-type': content_type})

                else:
                    '''Returns requested field(s) for exact match'''
                    i=0
                    data_string=''
                    while i<len(data)-1:
                        data_string+=data[i]+','
                        i+=1
                    data_string+=data[i]

                    if related_ep ==None:

                        r = requests.get(f'{self.base_url}/api/{endpoint}/?{search_field}={criteria}&fields!={data_string}', 
                                         headers={**self._token_header, 
                                                  'content-type': content_type}) 
                    else: #cross-search

                        r = requests.get(f'{self.base_url}/api/{endpoint}/?{related_ep}__{search_field}!={criteria}&fields={data_string}', 
                                         headers={**self._token_header, 
                                                  'content-type': content_type}) 

            else:


                if data == None:
                    '''Containment test; returns all fields'''

                    if related_ep == None:
                        r = requests.get(f'{self.base_url}/api/{endpoint}/?{search_field}__icontains!={criteria}', 
                                         headers={**self._token_header, 
                                                  'content-type': content_type}) 
                    else: #cross-search
                        r = requests.get(f'{self.base_url}/api/{endpoint}/?{related_ep}__{search_field}__icontains!={criteria}', 
                                         headers={**self._token_header, 
                                                  'content-type': content_type})

                else:
                    '''Containment test; returns requested field(s)'''
                    i=0
                    data_string=''
                    while i<len(data)-1:
                        data_string+=data[i]+','
                        i+=1
                    data_string+=data[i]

                    if related_ep==None:
                        r = requests.get(f'{self.base_url}/api/{endpoint}/?{search_field}__icontains!={criteria}&fields={data_string}', 
                                         headers={**self._token_header, 
                                                  'content-type': content_type}) 
                    else: #cross-search
                        r = requests.get(f'{self.base_url}/api/{endpoint}/?{related_ep}__{search_field}__icontains!={criteria}', 
                                             headers={**self._token_header, 
                                                      'content-type': content_type})                       

        if r.ok: 
            print('GET: OK')
        else: 
            print('GET: FAILED')

        if r.ok and parse_json:
            resp_json = r.json()        
            # handle cases: no vs one vs many results
            count = resp_json.get('count')
            if count is None or count == 0:
                return r.json()
            elif count == 1: 
                print('Found one resource, returning dict')
                return resp_json['results'][0]
            elif count >= 1: 
                print(f"Found {resp_json['count']} resources, returning list of dicts)")
                return r.json()['results']
        print('Returning response object')   
        return r
    
    def list_endpoints(self):
        return self.get()

First import the API Client.

In [ ]:
import escalateclient
from escalateclient import ESCALATEClient

Enter your ESCALATE username and password, and the port number at which the server is running. Then run the following cell. This will create an instance of the API Client and set up a token to interact with the REST API.

In [2]:
username= 'nsmina'
password= 'password11'
port='8000'

escalate = ESCALATEClient(
    f'http://localhost:{port}',
    username,
    password
)

To see all the model endpoints (this is also a good way to verify that you are connected):

In [3]:
escalate.list_endpoints()

GET: OK


{'action': 'http://localhost:8000/api/action/',
 'actiondef': 'http://localhost:8000/api/action-def/',
 'actionunit': 'http://localhost:8000/api/action-unit/',
 'actor': 'http://localhost:8000/api/actor/',
 'basebommaterial': 'http://localhost:8000/api/base-bom-material/',
 'billofmaterials': 'http://localhost:8000/api/bill-of-materials/',
 'bomcompositematerial': 'http://localhost:8000/api/bom-composite-material/',
 'bommaterial': 'http://localhost:8000/api/bom-material/',
 'calculation': 'http://localhost:8000/api/calculation/',
 'calculationdef': 'http://localhost:8000/api/calculation-def/',
 'condition': 'http://localhost:8000/api/condition/',
 'conditiondef': 'http://localhost:8000/api/condition-def/',
 'experiment': 'http://localhost:8000/api/experiment/',
 'experimentinstance': 'http://localhost:8000/api/experiment-instance/',
 'experimenttemplate': 'http://localhost:8000/api/experiment-template/',
 'experimenttype': 'http://localhost:8000/api/experiment-type/',
 'experimentwork

## Searching the database

To search the database for an existing material (or action, or workflow, or any endpoint)... enter the endpoint below. Make sure it is one of the existing endpoints in the database. 

Choose a field by which to search this database. For instance, you can look up a material by its description (which corresponds to the name of the material). Enter that as well. Then enter your search criteria and whether it is exact (True or False). 

Suppose that we want to look up Lead Diiodide and return all the exact matches. Note that exact match searches are case-sensitive, so if you did not capitalize the first letter of each word in the name, you would get no results.

In [5]:
escalate.search( 
        endpoint='material', #we are looking in the material database table
        related_ep=None,
        search_field='description', #we are searching by description
        criteria= 'Lead Diiodide', #we want the description of the material to be 'Lead Diiodide'
        data=None, 
        exact=True, #exact matches only
        negate=False,
        parse_json=True, 
        content_type='application/json')

GET: OK
Found one resource, returning dict


{'url': 'http://localhost:8000/api/material/28fe860b-44ae-4737-9fc9-b1714facc07d/',
 'uuid': '28fe860b-44ae-4737-9fc9-b1714facc07d',
 'edocs': [],
 'tags': [],
 'notes': [],
 'property': [],
 'add_date': '2021-08-31T18:59:22.626532',
 'mod_date': '2021-08-31T18:59:22.626551',
 'description': 'Lead Diiodide',
 'consumable': True,
 'material_class': 'model',
 'internal_slug': 'lead-diiodide',
 'status': 'http://localhost:8000/api/status/0547fd81-cea5-4413-aa50-684c4624a503/',
 'actor': None,
 'identifier': ['http://localhost:8000/api/material-identifier/125b2798-2216-4f9b-9671-070ede6e1620/',
  'http://localhost:8000/api/material-identifier/d240f536-0388-4d48-8678-4ab678b8f226/',
  'http://localhost:8000/api/material-identifier/e44f7760-3383-4aa2-b101-d80a8bb4a513/',
  'http://localhost:8000/api/material-identifier/b6686937-f3d3-4112-89dd-29e606414166/',
  'http://localhost:8000/api/material-identifier/545f1643-f0b8-46ec-83b4-eda59f826b6c/',
  'http://localhost:8000/api/material-identifi

Note this returns every single field associated with Lead Diiodide. Maybe we just want the url and the description. We would enter these, in list form, as data, and then run our search again.

In [6]:
escalate.search(endpoint='material',
        related_ep=None, 
        search_field='description',
        criteria= 'Lead Diiodide',
        data=['url', 'description'], #specify the fields for the search to return, in list form
        exact=True,
        negate=False,
        parse_json=True, 
        content_type='application/json')

GET: OK
Found one resource, returning dict


{'url': 'http://localhost:8000/api/material/28fe860b-44ae-4737-9fc9-b1714facc07d/',
 'description': 'Lead Diiodide'}

Suppose we instead want to know how many materials in the inventory contain the word "lead" in their names. We would change our criteria, and also change exact to False. Note that the search criteria is no longer case sensitive.

In [7]:
escalate.search(endpoint='material',
        related_ep=None, 
        search_field='description',
        criteria= 'lead', #not case-sensitive
        data=['url', 'description'], 
        exact=False, #the search will return anything that contains "lead" in the name
        negate=False,
        parse_json=True, 
        content_type='application/json')

GET: OK
Found 3 resources, returning list of dicts)


[{'url': 'http://localhost:8000/api/material/28fe860b-44ae-4737-9fc9-b1714facc07d/',
  'description': 'Lead Diiodide'},
 {'url': 'http://localhost:8000/api/material/dccde9ad-1b5b-4433-af52-412d02362b38/',
  'description': 'Lead(II) bromide'},
 {'url': 'http://localhost:8000/api/material/bdb2b329-6f54-4228-a0f4-c5facb4bc36f/',
  'description': 'Lead(II) acetate trihydrate'}]

If we want to search by a property other than name, we must do a cross-search within the material-identifier table. Let's demonstrate using the INCHI key for lead diiodide. Note this requires specifying the endpoint of the material identifier table `(related_ep)`.

In [8]:
escalate.search( endpoint='material',
        related_ep='identifier', #endpoint of related table in which we are searching 
        search_field='description',
        criteria= 'RQQRAHKHDFPBMC-UHFFFAOYSA-L', #inchi key
        data=['url', 'description'],
        exact=False,
        negate=False,
        parse_json=True, 
        content_type='application/json')

GET: OK
Found one resource, returning dict


{'url': 'http://localhost:8000/api/material/28fe860b-44ae-4737-9fc9-b1714facc07d/',
 'description': 'Lead Diiodide'}

Lastly, we have the option to search for all entries within a data table that do NOT contain the specified criteria. Let's demonstrate this using the Action endpoint. Here we find all actions that do not contain the word "heat" in their description.

In [9]:
escalate.search( 
        endpoint='action',
        related_ep=None, 
        search_field='description',
        criteria= 'heat',
        data=['description'], #let's return the descriptions only, for readability
        exact=False,
        negate=True, #this makes the search return everything that DOES NOT contain 'heat'
        parse_json=True, 
        content_type='application/json')

GET: OK
Found 184 resources, returning list of dicts)


[{'description': 'Dispense Metal Stock:  -> Sample Prep Plate: Plate well#: B2'},
 {'description': 'Dispense Resin: Resin -> Resin Plate: Plate well#: A1'},
 {'description': 'Dispense Resin: Resin -> Resin Plate: Plate well#: A2'},
 {'description': 'Dispense Resin: Resin -> Resin Plate: Plate well#: A3'},
 {'description': 'Dispense Resin: Resin -> Resin Plate: Plate well#: A4'},
 {'description': 'Dispense Resin: Resin -> Resin Plate: Plate well#: A5'},
 {'description': 'Dispense Resin: Resin -> Resin Plate: Plate well#: A6'},
 {'description': 'Dispense Resin: Resin -> Resin Plate: Plate well#: B1'},
 {'description': 'Dispense Resin: Resin -> Resin Plate: Plate well#: B2'},
 {'description': 'Dispense Sample H2O: H2O -> Sample Prep Plate: Plate well#: A1'},
 {'description': 'Dispense Sample H2O: H2O -> Sample Prep Plate: Plate well#: A2'},
 {'description': 'Dispense Sample H2O: H2O -> Sample Prep Plate: Plate well#: A3'},
 {'description': 'Dispense Sample H2O: H2O -> Sample Prep Plate: P

#### GET - for simple, fast searches

Alternatively, for simple searches where we might not want to worry about setting all the filters, we can just use `Get`. It takes an endpoint, search field, criteria, and a list of data fields to return

In [10]:
escalate.get(endpoint='material',
            data={'description': 'Formic Acid', #data={search field: criteria
                  'fields': ['url', 'description']}) #,fields to return}

GET: OK
Found one resource, returning dict


{'url': 'http://localhost:8000/api/material/597e1e3d-aea7-4f62-8636-9863ee4f86cf/',
 'description': 'Formic Acid'}

## Posting new data

To add a new material to the inventory, we use `Post`. For this function, the data is entered as a dictionary, with keys corresponding to field names and items corresponding to the entries of those fields. You can choose which fields to fill in data for. 

Let's post ammonium iodide, NH4. We will populate the description, material class, and the fact that it is consumable.

In [ ]:
r= escalate.post(endpoint='material/', 
              data={'description': 'Ammonium Iodide', 
                    'material_class': 'model',
                    'consumable': True} 
                     )

r

Suppose we accidentally run the above cell twice - this will duplicate the ammonium iodide entry in the materials inventory. We can then make use of `delete` to remove one of the entries - we simply pass in the argument of the url of the entry that we want to remove:

In [ ]:
escalate.delete(r['url'])

In [ ]:
#re-run this cell if you deleted the entry, so that NH4I exists (once) in the inventory for purposes of the rest of the demo

escalate.post(endpoint='material/', 
              data={'description': 'Ammonium Iodide', 
                    'material_class': 'model',
                    'consumable': True} 
                     )

## Associating data across tables

For a new entry that we post, we might want to also add identifiers like the chemical formula and/or the INCHI key to the material_identifiers table, so that the entry is more complete and it's easier to find in future searches.

First we have to find the descriptions of these identifiers.

In [16]:
ids = {'InChIKey': 'UKFWSNCTAHXBQN-UHFFFAOYSA-N', 'Molecular_Formula': 'NH4I'}

def_urls={}

#use a loop to search for all the identifiers at once, and then store all their URLs in a dictionary

for i in ids.keys():
    r = escalate.get(endpoint='material-identifier-def',
                     data={'description': i, #data={search field: criteria
                      'fields': ['url']}) #,fields to return}
    def_urls[i]=r['url']
    
def_urls

GET: OK
Found one resource, returning dict
GET: OK
Found one resource, returning dict


{'InChIKey': 'http://localhost:8000/api/material-identifier-def/881e02c5-36ec-4034-8a1e-15be1bb9f45f/',
 'Molecular_Formula': 'http://localhost:8000/api/material-identifier-def/8e312939-ddc4-4bfa-b1a6-d90d1d5f4feb/'}

Now we have a dictionary containing the urls of the material identifier definitions that we want to associate with our specific material, Ammonium Iodide. We have to create instances of each of these definitions. The endpoint is material-identifier.

In [18]:
for key,val in def_urls.items():
    escalate.post(endpoint='material-identifier/', 
                  data={'description': ids[key], 
                        'material_identifier_def': val} 
                         )

<Response [201]>
POST: OK, returning new resource dict
<Response [201]>
POST: OK, returning new resource dict


Let's check if this worked.

In [24]:
id_urls={}

for key, val in ids.items():    #loop across the list containing the material identifier descriptions we just posted
    
    r = escalate.get(endpoint='material-identifier',
                         data={'description': val, 
                          'fields': ['description', 'url']})
    id_urls[key] = r['url']
    
    print(r)


GET: OK
Found one resource, returning dict
{'url': 'http://localhost:8000/api/material-identifier/a45582ee-9dac-42c3-be0a-94a2eaa3c7ed/', 'description': 'UKFWSNCTAHXBQN-UHFFFAOYSA-N'}
GET: OK
Found one resource, returning dict
{'url': 'http://localhost:8000/api/material-identifier/e03bb825-b43e-42ed-97b4-614154cfc30f/', 'description': 'NH4I'}


In [25]:
id_urls

{'InChIKey': 'http://localhost:8000/api/material-identifier/a45582ee-9dac-42c3-be0a-94a2eaa3c7ed/',
 'Molecular_Formula': 'http://localhost:8000/api/material-identifier/e03bb825-b43e-42ed-97b4-614154cfc30f/'}

Success! And if we were to navigate to each url, we'd see an associated material identifier definition url. So for our instance of the molecular formula, there is the url that corresponds to the material identifier definition for molecular formula.

Lastly, we need to associate each of these identifiers with the material (at the material endpoint). To do so, we use `Patch`, which will update the identifier field of our material entry.

In [27]:
#obtain url for the material using Get
mat_url = escalate.get(endpoint='material',
            data={'description': 'Ammonium Iodide', 
                  'fields': ['url', 'description']})['url']

mat_url

GET: OK
Found one resource, returning dict


'http://localhost:8000/api/material/d240c1f3-4f71-41b7-922b-370abe60e710/'

In [34]:
for val in id_urls.values():
    r = escalate.patch(url=mat_url, 
                 data={'identifier': val                 
                      })
    print(r)

<Response [200]>
<Response [200]>


**issue: as of now, each patch request overwrites the previous one**